In [47]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("roberta-base-openai-detector")

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base-openai-detector", torchscript=True
).eval()

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [57]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [58]:
model.config.id2label

{0: 'Fake', 1: 'Real'}

In [48]:
text = (
    """I am a person who lives in rural CO, I like living in rural areas, probably because when I was little I grew up in rural VT. 
I am 33 years old, I have a dog namd Dusty who I walk multiple times each day. Today I ran to the fire station to do the pack test then ran home. 
It started snowing today so I started a fire in the wood stove which is heating the house nicely as it is supposed to get quite cold this evening.

When I was in middle school I moved to The Netherlands and went to school there (International school of Amsterdam), I had to ride my bike there every
day, it was almost 4 km each way. While at the time it was tough for me to say goodbye to my friends from VT (the only I'd ever known), it was, upon
reflection an immensely valuable life experience to see how other people and other cultures lived. I was also able to pick up een beetje Nederlands!"""
    * 100
)
encoded_input = tokenizer(text, truncation=True, padding=True, return_tensors="pt")

In [51]:
%time
with torch.no_grad():
    (output,) = model(**encoded_input)
# output
probabilities = output.softmax(dim=1)
predicted_class = probabilities.argmax(dim=1).item()
predicted_class

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs


1

In [24]:
probabilities = output.logits.softmax(dim=1)
predicted_class = probabilities.argmax(dim=1).item()
predicted_class

1

In [36]:
example_input = encoded_input
from transformers import BertModel, BertTokenizer, BertConfig
import torch

enc = BertTokenizer.from_pretrained("bert-base-uncased")
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = enc.tokenize(text)

# Masking one of the input tokens
masked_index = 8
tokenized_text[masked_index] = "[MASK]"
indexed_tokens = enc.convert_tokens_to_ids(tokenized_text)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Creating a dummy input
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
dummy_input = [tokens_tensor, segments_tensors]

traced_model = torch.jit.trace(model, dummy_input)

In [56]:
# optimized_model = traced_model.optimize()

torch.jit.save(traced_model, "traced_model.pt")

AttributeError: 'RecursiveScriptModule' object has no attribute 'optimize'

In [55]:
%time
with torch.no_grad():
    output = traced_model(encoded_input)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.05 µs


RuntimeError: forward() Expected a value of type 'Tensor' for argument 'input_ids' but instead found type 'BatchEncoding'.
Position: 1
Value: {'input_ids': tensor([[    0,   100,   524,    10,   621,    54,  1074,    11,  3826,  6247,
             6,    38,   101,  1207,    11,  3826,   911,     6,  1153,   142,
            77,    38,    21,   410,    38,  2307,    62,    11,  3826, 37599,
             4,  1437, 50118,   100,   524,  2357,   107,   793,     6,    38,
            33,    10,  2335,   295, 45594, 25953,   219,    54,    38,  1656,
          1533,   498,   349,   183,     4,  2477,    38,  2075,     7,     5,
           668,  1992,     7,   109,     5,  6356,  1296,   172,  2075,   184,
             4,  1437, 50118,   243,   554,  1958,   154,   452,    98,    38,
           554,    10,   668,    11,     5,  5627, 25901,    61,    16, 11545,
             5,   790, 15481,    25,    24,    16,  3518,     7,   120,  1341,
          2569,    42,  1559,     4, 50118, 50118,  17